In [2]:
!pip install web3
import json
from web3 import Web3
import pandas as pd
import numpy as np
import networkx as nx 
from tqdm.notebook import tqdm
import math
import itertools
import random

You should consider upgrading via the 'c:\users\fedem\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
# StackOverflow ♥
# we need to filter out all those paths that ping-pongs during the path 
def findPaths(G,u,n):
    if n==0:
        return [[u]]
    paths = [[u]+path for neighbor in G.neighbors(u) for path in findPaths(G,neighbor,n-1)]
    return paths

def find_cycles(G,u,n):
    paths = findPaths(G,u,n)
    return [tuple(path) for path in paths if (path[-1] == u) and sum(x ==u for x in path) == 2]

def find_cycles_not_PingPong(G,u,n):
    paths = findPaths(G,u,n)
    return [tuple(path) for path in paths if (path[-1] == u) and sum(x ==u for x in path) == 2 and sum(prev==next for prev,cur,next in zip([None]+path[:-1], path, path[1:]+[None]))==0] #1) check that the last element is the starting element; 2) check that the starting element is counted just twice in the path; 3) check that previous element of the path is different from the subsequent one

In [4]:
def saveJson(path,file):
    with open(path, 'w') as f:
        json.dump(file, f)

In [5]:
with open(f"endpoints/baseTokens.json") as f:
    info_json = json.load(f)
_baseToken = info_json
# Adding several dexes increase a lot the overall tokens but most of all the overall paths, since the paths are given as permutations (cartesian product), of 
# each pool involved for a same multi-swap
#hence, there is a trade-off between the additional dex added and the additional constraint (minimum number of hedges/amount of reserves) to add for token selection
# better include peripheric tokens on a little amount of dexes or just well-connected tokens on several amount of dexes?
chain = "Celo"
dexes = ["Ubeswap"]
_testingQuantity = 0.61512 #increase this quantity in order to cut out all the pools with poor reserves
_wEth = _baseToken[chain]
thresholdNumber = 6 #minimum number of pools (=edges) → putting an high value you will consider just the "most connected" tokens (central parth of the associated oriented graph)
tokensInvolvedInCycle = 3 #Put 3 for triangular abitrage
showGraphs = False
onlyDifferentDexes = True

_userAccount = "0xD59488d45304b2C7DC593f491F7206073602854B"

In [6]:
pools = []
for i in range(len(dexes)):
    with open(f"pools/Pools{dexes[i]}.json") as f:
        info_json = json.load(f)
    pools.append(list(info_json))

allTokens = np.unique(np.concatenate(list(map(lambda x:np.array(list(map(lambda y:y["tokens"],x))).flatten(),pools))).ravel())

poolMatrix = {}
for i in tqdm(range(0,len(allTokens))): #iterates over the i-th token
    obj = {}
    for j in range(0,len(pools)): #iterates over the j-th dex 
        for z in range(len(pools[j])): #iterates over the z-th pool of the j-th dex
            if(allTokens[i] in pools[j][z]["tokens"])==True:
                otherIndex = 1 if pools[j][z]["tokens"].index(allTokens[i])==0 else 0
                if not (pools[j][z]["tokens"][otherIndex] in obj):
                    obj[pools[j][z]["tokens"][otherIndex]] =[]
                obj[pools[j][z]["tokens"][otherIndex]].append({pools[j][z]["indirizzoPool"]:dexes[j]})
    poolMatrix[allTokens[i]]=obj
#saves data as 
#i-thToken{
#   otherToken:[{poolAddress1:j-thDex},{poolAddressN:(j-th+N)Dex}] → in this way the pool is overwritten tho
# }


  0%|          | 0/248 [00:00<?, ?it/s]

In [7]:
dataframe = pd.DataFrame(poolMatrix)
dataframe = dataframe.fillna(0)
dataframe

,0x00400FcbF0816bebB94654259de7273f4A05c762,0x00Be915B9dCf56a3CBE739D9B9c202ca692409EC,0x025E09a03FC4a19b6Dd7c7d8d83BFDA82E510ED5,0x035eE610693a29Cb77fD6eFBEb9d9d278703e145,0x041954F3f34422aF8d1f11fD743f3A1B70C30271,0x059611daEdBA5Fe0875aC7c76d7cE47FfE5c39C5,0x0631DD22cd83508f50ea11e1CAE82801Bb9904aC,0x0DbD6958DBC84382ec3Dfdcc4FE068D3D7f3bF80,0x0Fe15362017b91A1D7c904b981Cc369FE28c4A89,0x0a4ff5C9BF93173BD5dAa6f4425942Fe643706ED,...,0xeD7160273E3f64183383B958a4140A394349D213,0xeb966c6AE397f28de1723A4a37Baa0F3da86c867,0xef4229c8c3250C675F21BCefa42f58EfbfF6002a,0xef97378E905D72dE06893D699D37c83061Ea9dB8,0xefa2515b0aFA7BF28F37Cd71019238c914D2B9fa,0xf1E0Cd564A51cE5902b0787C817F38B58E8B7D27,0xf212Fe874b02F5fbe918dc13b33501dcdfaA7111,0xf581dB2408F74267C8C8d3489A35535736E09186,0xf7Ad12D83013BC2C5c93Fab5bFAB5E57F6203b72,0xfCc6CeDf0559064D8E24014cc2151170657363a2
0x00Be915B9dCf56a3CBE739D9B9c202ca692409EC,[{'0x573bcEBD09Ff805eD32df2cb1A968418DC74DCf7'...,0,0,[{'0x911923b25c6Ca2B916da8005F0554B365d7dDf33'...,0,0,0,0,[{'0x0dB5c89780441781FbF887a2EA7AFf5691A71b0A'...,0,...,[{'0x354D379035b6018d1ff02eAa799e8cBf96076610'...,0,0,0,0,0,0,0,0,0
0x471EcE3750Da237f93B8E339c536989b8978a438,[{'0x0931738d2C2998173058B635524e437172a6a96C'...,[{'0xe7B5AD135fa22678F426A381C7748f6A5f2c9E6C'...,[{'0x606a73fe860D08E835687DF5aeF882cB5730d0F9'...,[{'0xf8E3891a38016dE2E6Ed5Fcc433Ca2BDB5e8f90c'...,[{'0x68f5ad364Aa0fc13a5FAdE9Ddf6BadA7DFCB026c'...,[{'0xe5EFBF323ECbc153546dEA64948cF6159aB282C8'...,[{'0x3AAd0a9D0DF37410a08Ac97ad872defD413B73a2'...,0,0,[{'0xB16e3fA5C55E8239e187aAbE2610ceB23a991340'...,...,[{'0x4E3B9d2Df2FB1aa6a6011f966770fDBA909e9dC3'...,[{'0xEcc0b72D9286273472b97FF37851EB341b7D3706'...,[{'0x9a23A61A8988fAA730A1Af09EaD45a993d7f5f3C'...,[{'0xA00d987bD21Ef8ebc96E3A577B9b5852351b9520'...,[{'0x7BEB9fdB0490175491DBD8403914899B4bd2FdEA'...,0,0,[{'0x52a7945350b197575BD7D5D889569D818D9d7685'...,0,[{'0x5C62f0A1f1ECc8F54bCf7BC92624e2809B0a44cD'...
0x765DE816845861e75A25fCA122bb6898B8B1282a,[{'0xcc90d7b497eC7D3a0C086D1B804BbA8A4a8e747C'...,[{'0x3dBE84b0445e2862D883c885e47bf618A8dB44a7'...,0,[{'0x6328A72067865da05D4B76f1380420E564edaa7d'...,[{'0x7Fba917c46eca341b6633B573e3F5BB7A6639C14'...,0,[{'0xd56FB0A18f7b0077d76d148e567f311Ad7935E9f'...,[{'0x416719EAe60Dbae7a0565E9010c4CcAFBC3e6D74'...,0,0,...,0,0,[{'0x8c774d1E374c08BbDBD5515cd6B063d0e5Cc110A'...,[{'0x124C3034D68AEdd9daE92Fcfc0bDf58f16Ab0B3c'...,0,0,0,0,0,0
0x452EF5a4bD00796e62E5e5758548e0dA6e8CCDF3,[{'0x704021C8799d8EaA366606808b912e8a9aB14ccb'...,[{'0x5f5a52C9D3CA54D9804B60a1af1eC9D5cCcd2d74'...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x73a210637f6F6B7005512677Ba6B3C96bb4AA44B,[{'0xa14c043eE8ea6FE9624A67FB84b7224428944f5e'...,[{'0x4550aa33f162A2887e40673Da1Bc8c23113Ce1F7'...,0,[{'0x7122f5B38479e34296Fa6D0B137625c5620C1819'...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0xCC8C3cd3E3C29BEdC471830E8455488937Cf74Ef,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0xf7Ad12D83013BC2C5c93Fab5bFAB5E57F6203b72,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,[{'0x66510D2F02Fa3f08925B6bf80d189fD61C7a0879'...,0,0,0,0
0x22aA4D78ce26553A8Be71E54448dbDc00494795c,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[{'0x5C8dcaDE49Fc1E065363417459aEe3B92be9D95d'...,0,0,0
0xf1E0Cd564A51cE5902b0787C817F38B58E8B7D27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[{'0x66510D2F02Fa3f08925B6bf80d189fD61C7a0879'...,0


In [8]:
arrayMatrix = dataframe.values
h, w = arrayMatrix.shape
G = nx.DiGraph()
for i in tqdm(range(h)): 
 for j in range(w):
   if arrayMatrix[i][j]!=0: 
     G.add_edge(i,j) 
import matplotlib.pyplot as plt 
plt.figure(5,figsize=(30,30))
ftmIndex = list(dataframe.columns).index(_wEth) #we point out node associated to FTM
color_map = ['red' if node == ftmIndex else '#1f78b4' for node in G]

  0%|          | 0/248 [00:00<?, ?it/s]

<Figure size 2160x2160 with 0 Axes>

In [9]:
if(showGraphs):
  nx.draw( G, node_size =50, node_color = color_map) 
  plt.show()

In [10]:
#We reduce the matrix to those tokens which have at least two different pools, eliminating recursively rows and columns
#In this way we eliminate nodes on the boundary and nodes that have very few links (=pools) with other tokens
filteredDataframe = dataframe
previousShape = (0,0)
while(filteredDataframe.shape!=previousShape):
    print(filteredDataframe.shape,previousShape)
    previousShape = filteredDataframe.shape
    data = (filteredDataframe[(filteredDataframe != 0).sum(1) >= thresholdNumber])
    data = data[data.index]#pd.DataFrame(poolMatrixReduced).loc[pd.DataFrame(poolMatrixReduced).columns]
    filteredDataframe = data

print(filteredDataframe.shape)
# filteredDataframe.to_json(f"Pools{dex}Filtered.json")

(248, 248) (0, 0)
(45, 45) (248, 248)
(42, 42) (45, 45)
(42, 42)


In [11]:
filteredDataframe

,0x00Be915B9dCf56a3CBE739D9B9c202ca692409EC,0x471EcE3750Da237f93B8E339c536989b8978a438,0x765DE816845861e75A25fCA122bb6898B8B1282a,0x452EF5a4bD00796e62E5e5758548e0dA6e8CCDF3,0x73a210637f6F6B7005512677Ba6B3C96bb4AA44B,0x47264aE1Fc0c8e6418ebe78630718E11a07346A8,0x37191488FbEe23652eC7794Cfb81020528968C7d,0x17700282592D6917F6A73D0bF8AcCf4D578c131e,0x94140c2eA9D208D8476cA4E3045254169791C59e,0x918146359264C492BD6934071c6Bd31C854EDBc3,...,0x39d6477522eB543d750AF82537325FB2930c1Aa6,0x39049C02A56c3ecd046F6c2A9BE0CffA2bc29C08,0xF06768797ba4e86abFa5aDFBFD223742F2657960,0x3dF39266F1246128C39086E1b542Db0148A30d8c,0x1E593F1FE7B61c53874B54EC0c59FD0d5eb8621e,0x041954F3f34422aF8d1f11fD743f3A1B70C30271,0xE685d21b7B0FC7A248a6A8E03b8Db22d013Aa2eE,0xE919F65739c26a42616b7b8eedC6b5524d1e3aC4,0xA649325Aa7C5093d12D6F98EB4378deAe68CE23F,0xD15EC721C2A896512Ad29C671997DD68f9593226
0x00Be915B9dCf56a3CBE739D9B9c202ca692409EC,0,[{'0xe7B5AD135fa22678F426A381C7748f6A5f2c9E6C'...,[{'0x3dBE84b0445e2862D883c885e47bf618A8dB44a7'...,[{'0x5f5a52C9D3CA54D9804B60a1af1eC9D5cCcd2d74'...,[{'0x4550aa33f162A2887e40673Da1Bc8c23113Ce1F7'...,[{'0x218AEd024d51EcBd6a82D143be04Cd25d563628e'...,[{'0x6c52401217fF21D6e37fFeb2066D03de35ca4B67'...,[{'0x0400cbCD2Bf5835949be4b4A74a37c73254345fa'...,[{'0x12E42ccf14B283Ef0a36A791892D18BF75Da5c80'...,[{'0x70fc1242b727b11c7aAD8088aC2d1d0c959e6023'...,...,0,0,0,0,0,0,0,0,0,0
0x471EcE3750Da237f93B8E339c536989b8978a438,[{'0xe7B5AD135fa22678F426A381C7748f6A5f2c9E6C'...,0,[{'0x1E593F1FE7B61c53874B54EC0c59FD0d5eb8621e'...,[{'0x0c672CF5131eB81D1b141A0C1c9631e751E3c608'...,[{'0x0B81CF47C8F97275d14C006E537D5101b6C87300'...,[{'0x2994297c01A441097535781731711a8122139B06'...,[{'0x98b88dD1a0187cEDffb94b983c10bDCaa62e652B'...,[{'0x19a75250c5A3ab22a8662e55a2B90FF9d3334B00'...,[{'0x19af88F2bB4283Faa95879AE8E8BD6B6C98b4F64'...,[{'0xb460F9Ae1fEa4f77107146C1960bb1C978118816'...,...,[{'0x7B7b05A80b6041cD039C4e487f269f47A776e396'...,[{'0xa2e63c3b9D027270Ff5892A552aaac50515ee61D'...,[{'0x5724CcCFef36B20250817Aa28E0D68707bDd0D9A'...,[{'0xA413103064E6Fb32744f39Cf3B9D16168ded7590'...,[{'0x02dbe5Ff9503A72c258eAcA5f2A76e4AFE472051'...,[{'0x68f5ad364Aa0fc13a5FAdE9Ddf6BadA7DFCB026c'...,[{'0x256762736c4D1E7c6F39B00960D5eAc81446FB64'...,[{'0xaE9A62F50171B3E1C4b3345D3638764Ecf1A78a6'...,[{'0x2004d4b6edeA896433B9614031caeC7412440862'...,[{'0x9c895B5edb4a07CDF16cD0e62356E757F4C11C52'...
0x765DE816845861e75A25fCA122bb6898B8B1282a,[{'0x3dBE84b0445e2862D883c885e47bf618A8dB44a7'...,[{'0x1E593F1FE7B61c53874B54EC0c59FD0d5eb8621e'...,0,[{'0x2e6F82359d24598F0143B5Cc8edaa1D32be0d75d'...,[{'0x1Eb738eC1D46c9BEfE95e830E19d0f537619F2D7'...,0,[{'0xfe801495eaAc62257a1258652a3e3c923B61C45d'...,[{'0x1657D8e20B7b64E4a1a0F4eeaF3E65943D9d4e69'...,[{'0x745828A7964091b18098F49d07BfB2bC3d993E8F'...,[{'0x4857E2F4e0b68bD31149f3f44CE0d883B7c4E440'...,...,[{'0xc5b530C5d36c34966A3A667Bed6D71D958B484aB'...,[{'0x883d42f6Cb9Ee9a69E2d68d9709DEbCcf45BC5e8'...,[{'0x13ce4Bff747F47b536DF29C86C1299e8B00cAe2d'...,[{'0x7d808c99Ee93eED012C0110dB9874AB469fD66EC'...,[{'0xb01049763CF2584aedd0601Fc3509DDB97a5cE3f'...,[{'0x7Fba917c46eca341b6633B573e3F5BB7A6639C14'...,[{'0x254a3fA791B622badd36a8a65ea2FE0A7dad990A'...,[{'0xB08D69f18a94e08534Ffd3A6D06372B1823dFc47'...,0,[{'0x125D1DcA8957055d88fa80596bb538F53E5Ee599'...
0x452EF5a4bD00796e62E5e5758548e0dA6e8CCDF3,[{'0x5f5a52C9D3CA54D9804B60a1af1eC9D5cCcd2d74'...,[{'0x0c672CF5131eB81D1b141A0C1c9631e751E3c608'...,[{'0x2e6F82359d24598F0143B5Cc8edaa1D32be0d75d'...,0,[{'0x5326C288b431e8D01787114662c7B3c2eC730A39'...,0,[{'0x4045842B1bbD83BdA63ec5Ad58f86F24B5483278'...,[{'0x421D68FE79c557f2B1e9B3f2AD1D6983FaA17ec2'...,0,0,...,0,0,0,0,0,0,0,0,0,0
0x73a210637f6F6B7005512677Ba6B3C96bb4AA44B,[{'0x4550aa33f162A2887e40673Da1Bc8c23113Ce1F7'...,[{'0x0B81CF47C8F97275d14C006E537D5101b6C87300'...,[{'0x1Eb738eC1D46c9BEfE95e830E19d0f537619F2D7'...,[{'0x5326C288b431e8D01787114662c7B3c2eC730A39'...,0,0,[{'0x64FB872e416051450bC0478c914eC67E9eF961b5'...,[{'0xac9ab918A0F36C1Ff53f14E9CB49F81bE01b8

In [12]:
arrayMatrix = filteredDataframe.values
h, w = arrayMatrix.shape
G = nx.DiGraph()
for i in tqdm(range(h)): 
 for j in range(w):
  #  if arrayMatrix[i][j] !=0 and arrayMatrix[i][j] !="0x00": 
#    if "0x" in str(arrayMatrix[i][j]): 
   if arrayMatrix[i][j]!=0: 
      G.add_edge(i,j) 
import matplotlib.pyplot as plt 
plt.figure(5,figsize=(5,5))
ftmIndex = list(filteredDataframe.columns).index(_wEth) #we point out node associated to FTM
color_map = ['red' if node == ftmIndex else '#1f78b4' for node in G]

  0%|          | 0/42 [00:00<?, ?it/s]

<Figure size 360x360 with 0 Axes>

In [13]:
if(showGraphs):
   nx.draw( G,node_size=50, node_color=color_map ) 
   plt.show()

In [14]:
#We define all the cycles that are present in such a matrix: we don't need to do it everytime, it's enough doing it just one time and save the path involving FTM as first input token
#Once we collected the paths, we will ask the information of the pools involved and we will apply recursively (for each path found) the "theorem" of the paper regarding existence of exploitable arbitrages   
# https://42papers.co/pdf/2105.02784.pdf

ftmIndex = list(filteredDataframe.columns).index(_wEth) #for future work, getting confidence with flashloans, starting node may be any "flashloanable" token, not only FTM
cycles = find_cycles_not_PingPong(G,ftmIndex,tokensInvolvedInCycle) #we extract every cycle (=ordered tuple of nodes) starting from the InputToken and with a length equal to "tokensInvolvedInCycle"

tokenPaths = []
poolsPaths = []
dexesPaths = [] 

#For each cycle, we translate the ordered tuple of nodes in a ordered tuple of tokens and in a ordered tuple of pools (with length n-1) 
for j in tqdm(range(0,len(cycles))):
    currentCycle = cycles[j]
    cycleTokens=[]
    poolsInvolved = []
    dexesInvolved = []
    for i in range(0,len(currentCycle)):
        cycleTokens.append(filteredDataframe.columns[currentCycle[i]])
        if(i>0): #since dimension of token Path is n while dimension of poolpath (dexpath) is n-1
            valueOnMatrix = filteredDataframe[filteredDataframe.columns[currentCycle[i-1]]][filteredDataframe.columns[currentCycle[i]]]
            poolsForCurrentOtherToken = [] #here we will store all the pools associated to the i-th swap
            dexesOfPoolForCurrentOtherToken = [] #here we will store all the dexes associated to the matching index pools
            for z in range(len(valueOnMatrix)):
                poolsForCurrentOtherToken.append(list(valueOnMatrix[z].keys())[0])
                dexesOfPoolForCurrentOtherToken.append(valueOnMatrix[z][list(valueOnMatrix[z].keys())[0]])
            poolsInvolved.append(poolsForCurrentOtherToken)
            dexesInvolved.append(dexesOfPoolForCurrentOtherToken)
    #now poolsInvolved is an array of arrays: each array contains the pools for passing from i-th token to (i+1)-th token to i-th token
    #now we need to create all the combinations between the values of these subArrays
    #in other words, we create the cartesian Product of such sybarrays, we use itertools
    poolsInvolved = list(itertools.product(*poolsInvolved))
    dexesInvolved = list(itertools.product(*dexesInvolved))
    # Now, since poolsInvolved is still an array of arrays (but now each subarray is a permutation of pools),
    # we push all the paths in the global variables: of course in tokenPaths some elements will be redundant
    # simply becasue there will be several pool-paths associated to the same token-path (since more dexes are involved)
    for k in range(len(poolsInvolved)):
        tokenPaths.append(cycleTokens)
        poolsPaths.append(list(poolsInvolved[k])) #we convert the tuple in a list
        dexesPaths.append(list(dexesInvolved[k]))

  0%|          | 0/446 [00:00<?, ?it/s]

In [17]:
# Now we remove all those paths in which a same pool is repeated several times in the same path
#Moreover, we check if there is any issue in the arbitrage path by simulating a multiswap, in order to discard all those paths involving scam tokens

with open("abis/Lifeguard.json") as f:
    info_json = json.load(f)
abi = info_json["abi"]

with open("endpoints/deployedAddresses.json") as f:
    info_json = json.load(f)
contract = info_json

with open("endpoints/indirizziDex.json") as f:
    info_json = json.load(f)
dexAddresses = info_json

with open("endpoints/chainsEndpoint.json") as f:
    info_json = json.load(f)
chainsEndpoint = info_json

url = random.choice(chainsEndpoint[chain]) #we don't consider multi-chain swaps

# url = "http://localhost:7545"
_routers = [dexAddresses[dexes[i]]["router"] for i in range(len(dexes))]
indirizzoContrattoCustom = contract[chain]#Mainnet 
testingQuantity = math.floor(_testingQuantity*(math.pow(10,18)))

w3 = Web3(Web3.HTTPProvider(url))
contrattoLifeguard = w3.eth.contract(address=indirizzoContrattoCustom, abi=abi)

#START FROM HERE!

finalPoolsPaths = []  # we will apply the "theorem" iterating on this list
# finalRoutersPaths = []
finalRoutersPaths2 = []
finalTokenPaths = []
for i in tqdm(range(0, len(poolsPaths))):
    if(len(set(poolsPaths[i])) == len(poolsPaths[i])):
        goodPath = False
        routerPath = [dexAddresses[dexesPaths[i][j]]["router"] for j in range(len(dexesPaths[i]))]
        if(onlyDifferentDexes and len(set(routerPath))==len(routerPath)):
            continue
        try:
            #I need to write first the function for multiSwap on different dexes
            #it must take as input the token path (dimension "n") and the path of routers (dimension "n/2"==dimension of poolPath) 
            # in the for cycle, it invokes the approve function of the i-th router address

            # DA FARE: 
            # goodPath = True 
            # goodPath = contrattoLifeguard.functions.multipleSwapUniCelo(tokenPaths[i],_router,True).call({ "from": _userAccount, "value": testingQuantity, "gas":5000000 }) if chain=="Celo" else contrattoLifeguard.functions.multipleSwapUni(tokenPaths[i],_router,True).call({ "from": _userAccount, "value": testingQuantity, "gas":5000000 })
            # goodPath = contrattoLifeguard.functions.multipleSwapMultiUni(tokenPaths[i],routerPath,True).call({ "from": _userAccount, "value": testingQuantity, "gas":5000000 })
            goodPath = contrattoLifeguard.functions.multipleSwapOnlyPools(tokenPaths[i],poolsPaths[i],False,True).call({ "from": _userAccount, "value": testingQuantity, "gas":5000000 }) if chain=="Celo" else contrattoLifeguard.functions.multipleSwapOnlyPools(tokenPaths[i],poolsPaths[i],True,True).call({ "from": _userAccount, "value": testingQuantity, "gas":5000000 })
        
        except Exception as e: print(e)
        
        if(goodPath):
            finalPoolsPaths.append(poolsPaths[i])
            finalTokenPaths.append(tokenPaths[i])
            # finalRoutersPaths.append(dexesPaths[i])
            finalRoutersPaths2.append(routerPath)
print(len(finalTokenPaths))
with open(f'monitorUtils/finalPoolsPaths{"_".join(dexes)}.json', 'w') as f:
    json.dump(finalPoolsPaths, f)
with open(f'monitorUtils/finalTokenPaths{"_".join(dexes)}.json', 'w') as f:
    json.dump(finalTokenPaths, f)
# with open(f'monitorUtils/finalDexesPaths{"_".join(dexes)}.json', 'w') as f:
#     json.dump(finalRoutersPaths, f)
with open(f'monitorUtils/finalDexesPaths{"_".join(dexes)}.json', 'w') as f:
    json.dump(finalRoutersPaths2, f)
# finalTokenPaths

  0%|          | 0/446 [00:00<?, ?it/s]

execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted:

In [18]:
# finalPools1=filteredDataframe.to_numpy().flatten() 
finalPools1 = np.array(finalPoolsPaths).flatten() #pool addresses we will query on the smart contract (associated to tokens that have links at least with "thresholdNumber" other tokens)
finalPools = np.unique(finalPools1[finalPools1!=0])
with open(f'monitorUtils/snipedPools{"_".join(dexes)}.json', 'w') as f:
    json.dump(finalPools.tolist(), f)
print(len(finalPools))

204


C:\Users\Fedem\AppData\Local\Temp/ipykernel_19844/4025130058.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  finalPools = np.unique(finalPools1[finalPools1!=0])
